In [ ]:
# Imports
# import libraries for simple image plotting and 
import matplotlib.pyplot as plt
import numpy as np

import torchvision.transforms as transforms
import torchvision as torchvision

from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
from datetime import datetime
import pytz
import datetime
import random


tz = pytz.timezone('Europe/Berlin')

gc = gspread.authorize(GoogleCredentials.get_application_default())



In [ ]:
student_name = "yourName"
assert student_name != "yourName"

In [ ]:
#@title Result Form
gsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1KsS7ApUdWT1gRy20qKlRYMRV28ri0Gdb_TE-La_tD3Y/edit#gid=232421945")

def write_result(task_number, result=None):
  worksheet = gsheet.worksheet(f"task{task_number}")
  current_time = datetime.datetime.now(tz).strftime("%X")
  current_date = str(datetime.date.today())
  if result:
    worksheet.append_row([student_name, current_time, current_date, result])
    print(f"Task {task_number} successfully solved by {student_name} at {current_time} with result: {result}")
  else:
    worksheet.append_row([student_name, current_time, current_date])
    print(f"Task {task_number} successfully solved by {student_name} at {current_time}")

print("Reporting enabled - write_result(number_of_task, result='your result') ")


In [ ]:
# quick example for object oriented programming: working with paths (folders and files)
assert student_name != "yourName"
write_result(0)


# Data Augmentation

It is a common fact that medical data is scarce. But to learn a very good model, the network needs a lot of data. So to tackle the problem we perform data augmentation.

Data augmentation is a strategy that enables practitioners to significantly increase the diversity of data available for training models, without actually collecting new data. 

Data augmentation techniques such as cropping, padding, and horizontal flipping are commonly used to train large neural networks.

![Data Augmentation](https://cdn-images-1.medium.com/max/1000/1*C8hNiOqur4OJyEZmC7OnzQ.png)
[Source](https://cdn-images-1.medium.com/max/1000/1*C8hNiOqur4OJyEZmC7OnzQ.png) 



In [ ]:
!wget https://github.com/CS4MS/CS4MS_W21/raw/main/images/cat.jpg

from PIL import Image
cat = Image.open("cat.jpg")

plt.axis('off')
plt.imshow(cat)

In [ ]:
def imshow(img):
    npimg = img.numpy()
    fig, ax = plt.subplots(figsize=(20, 20))
    ax.axis('off')
    ax.imshow(np.transpose(npimg, (1, 2, 0)))

def denorm(img):
    img[0,:,:] = (img[0,:,:] * np.asarray(norm_std[0])) + np.asarray(norm_mean[0])
    img[1,:,:] = (img[1,:,:] * np.asarray(norm_std[1])) + np.asarray(norm_mean[1])
    img[2,:,:] = (img[2,:,:] * np.asarray(norm_std[2])) + np.asarray(norm_mean[2])
    return img


In [ ]:
augmentation = transforms.Compose([
                                  # resize image to the network input size
                                  transforms.Resize((224,224)),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomRotation(degrees=60),
                                  transforms.RandomCrop(180),
                                  transforms.ToTensor(),
                                   ])
# Complete list: https://pytorch.org/docs/stable/torchvision/transforms.html
# Examples: 
# torchvision.transforms.RandomErasing()
# torchvision.transforms.RandomAffine(degrees, translate=None, scale=None, shear=None, resample=False, fillcolor=0) --> transforms.RandomAffine(degrees=20, shear=[0,50]),
# torchvision.transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=0) --> transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
# transforms.RandomCrop(180),

In [ ]:
images = []
for i in range(16):
    temp_im = augmentation(cat)
    images.append(temp_im)
    
# show images
imshow(torchvision.utils.make_grid(images))

In [ ]:
# Try out your own transformations 
task1_done=False
own_aug =  transforms.Compose([
                              transforms.Resize((224,224)),
                              transforms.ToTensor(),
                              ])

images = []
for i in range(16):
    temp_im = own_aug(cat)
    images.append(temp_im)  
# show images
imshow(torchvision.utils.make_grid(images))


if len(own_aug.transforms) <= 2:
  print("You have to do more than that!")
else:
  task1_done = True
  print("task1 done!")


# Done with Task1

Once you are happy with your augmentation submit your results



In [ ]:
effect = "write what the effect of your transformations is - if you want you can give it a score between 0-10"


if task1_done and effect != "write what the effect of your transformations is - if you want you can give it a score between 0-10":
  effect += "\n"+str(own_aug)
  write_result(1, effect)
else:
  print("you didnt solve task1 yet.")

## Normalization
Data normalization is an important step which ensures that each input parameter (pixel, in this case) has a similar data distribution. This makes convergence faster while training the network. 

Data normalization is done by subtracting the mean from each pixel and then dividing the result by the standard deviation. The distribution of such data would resemble a Gaussian curve centered at zero. 

Since, skin lesion images are natural images, we use the normalization values (mean and standard deviation) from [Imagenet dataset.](http://www.image-net.org/)
*norm_mean = (0.4914, 0.4822, 0.4465)*

*norm_std = (0.2023, 0.1994, 0.2010)*

This denotes mean and standard deviation for each channel(RGB) of an image.


We perform following data augmentation:
- Resize the image.
- Flipping the image horizontally.
- Randomly rotating image.
- Normalizing the image.

In [ ]:

# Imagenet values
norm_mean = (0.4914, 0.4822, 0.4465)
norm_std = (0.2023, 0.1994, 0.2010)

# define the transformaitons the images go through each time it is used for training
# includes augmentation AND normalization as descirbed above
augmentation_train = transforms.Compose([
                                  # resize image to the network input size
                                  transforms.Resize((224,224)),
                                  # randomly perform a horizontal flip of the image
                                  transforms.RandomHorizontalFlip(),
                                  # rotate the image with a angle from 0 to 60 (chosen randomly)
                                  transforms.RandomRotation(degrees=60),
                                  # convert the image into a tensor so it can be processed by the GPU
                                  transforms.ToTensor(),
                                  # normalize the image with the mean and std of ImageNet
                                  transforms.Normalize(norm_mean, norm_std),
                                   ])

In [ ]:
images = []
for i in range(16):
    temp_im = augmentation_train(cat)
    images.append(temp_im)  
# show images
imshow(torchvision.utils.make_grid(images))


# MedMNIST

https://github.com/MedMNIST/MedMNIST.git


In [ ]:
!git clone https://github.com/MedMNIST/MedMNIST.git

In [ ]:
import os
import sys
from tqdm import trange
from tqdm import tqdm
from skimage.util import montage
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

sys.path.insert(0, "/content/MedMNIST")
import MedMNIST.medmnist as medmnist
from MedMNIST.medmnist.dataset import PathMNIST, ChestMNIST, DermaMNIST, OCTMNIST, PneumoniaMNIST, RetinaMNIST, BreastMNIST, OrganMNISTAxial, OrganMNISTCoronal, OrganMNISTSagittal
from MedMNIST.medmnist.evaluator import getAUC, getACC
from MedMNIST.medmnist.info import INFO

In [ ]:
print("Version:", medmnist.__version__)

In [ ]:
data_flag = 'dermamnist'
download = True
input_root = '/content'

flag_to_class = {
    "pathmnist": PathMNIST,
    "chestmnist": ChestMNIST,
    "dermamnist": DermaMNIST,
    "octmnist": OCTMNIST,
    "pneumoniamnist": PneumoniaMNIST,
    "retinamnist": RetinaMNIST,
    "breastmnist": BreastMNIST,
    "organmnist_axial": OrganMNISTAxial,
    "organmnist_coronal": OrganMNISTCoronal,
    "organmnist_sagittal": OrganMNISTSagittal,
}

DataClass = flag_to_class[data_flag]

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])
label_dict = info['label']

print(f"Info:\n{info}\n")
print(f"Task:\n{task}\n")
print(f"Channels:\n{n_channels}\n")
print(f"Number of classes:\n{n_classes}\n")
print(f"Label:\n{label_dict}\n")


In [ ]:
# no augmentation for the test data only resizing, conversion to tensor and normalization
augmentation_test = transforms.Compose([
                    transforms.Resize((224,224)),
                    transforms.ToTensor(),
                    transforms.Normalize(norm_mean, norm_std),
                    ])

# Train, Test and Validation Split
It is a best practice to split the entire dataset into 3 parts:
- Train: Used to train a network.
- Validation: Fine tune the network.
- Test: Kept as unseen data to gauge the performance of out trained network.


In [ ]:
# load the data
train_dataset = DataClass(root=input_root, split='train', transform=augmentation_train, download=download)
test_dataset = DataClass(root=input_root, split='test', transform=augmentation_test, download=download)
val_dataset = DataClass(root=input_root, split='val', transform=augmentation_test, download=download)
nonorm_dataset = DataClass(root=input_root, split='train', transform=transforms.ToTensor(), download=download)


In [ ]:
print(f"Length of training dataset: {len(train_dataset)}")

In [ ]:
print(train_dataset)
print("===================")
print(val_dataset)
print("===================")
print(test_dataset)
print("===================")
print(nonorm_dataset)

Let's try to use the __getitem__ method of the ImageFolder class.

In [ ]:
# Check the dimension of the 1200th image in the nonorm_dataset and its corresponding label
image, label = nonorm_dataset[0]
print("Image Shape: {} \nLabel: {} \nLesion Type: {}".format(image.shape, label, label_dict[str(label[0])]))

In [ ]:
plt.axis('off')
image = image.permute(1, 2, 0)
plt.imshow(image)

## Task2

sample 16 different random images from the train_dataset
hint: you can use the function *random.sample(range(0,10), 5)*

save each image in the images list and print its index and label before adding it. Also add the label to the label_list.
hint: do it in a for loop

finally look at what you just created

In [ ]:
randomlist = #code here - hint: check out random.sample
print(randomlist)
label_list = [] #This has to be filled in the for loop
images = [] #This has to be filled in the for loop
for i in randomlist:
    temp_im, label = train_dataset[i]
    print(f"adding index: {i} with label {label}")
    ## code here - hint: append is a good choice here

# show images
imshow(torchvision.utils.make_grid(images))

## Submit Task2

In [ ]:
result2="Write here what you noticed when you looked at the visualized images and labels"

In [ ]:
if len(label_list) == 16 and len(images) == 16 and result2!="Write here what you noticed when you looked at the visualized images and labels":
  result2 += "\n" + "labels: " + str(label_list) + "\n" + "indices" + str(randomlist)
  write_result(2, result2)
else:
  print("something didnt go as expected, check if you solved the Task2!")


# Dataloader

In [ ]:
BATCH_SIZE = 16
# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
batch_images, batch_labels = next(iter(train_loader))

In [ ]:
# functions to show an image
fig = plt.figure(figsize=(20, 20))

def denorm_imshow(img):
    img = denorm(img)
    npimg = img.numpy()
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# show images
denorm_imshow(torchvision.utils.make_grid(batch_images))


# Model Inference
Now, we have our dataset loaded Let's try to use them as input to our model.

For now, we will use an pre trained network on a different task and do inference on the test set of out data. Let's see how is the performance without training.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# load a pretrained model
from torch import nn
import torchvision

net = torchvision.models.resnet18(pretrained = True)

# We replace last layer of resnet to match our number of classes which is 7
# more details next lecture
net.fc = nn.Linear(512, n_classes)
net = net.to(device)

In [ ]:
# counter for correct predictions
correct = 0
# counter for all predicted samples
total = 0

# set network to evaluation mode (next lecture)
net.eval()

# this is for next lecture..
with torch.no_grad():
  images, labels = next(iter(train_loader))
  labels = torch.reshape(labels,(1,-1)).squeeze()
  images, labels = images.to(device), labels.to(device)
  outputs = net(images)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct = (predicted == labels).sum()

print(labels)
print(predicted)

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

# Next time

Now that we have the dataloaders and augmentations we can finally train our network so that it can actually learn to identify skin leasions.